In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data  as data
from torch.utils.data import DataLoader
from torchsummary import summary
from torch import optim
from torch.optim.lr_scheduler import StepLR

from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import os

from torchvision import utils 
import matplotlib.pyplot as plt

import numpy as np
from torchsummary import summary
import time
import copy
from random import *
import math

import h5py
import cmath
import time
import scipy.io

In [2]:
num = 2 # complex number
m = 16 # number of vertical antennas in BS 
n = 16 # number of horizental antennas in BS
N_BS = m*n # total number of antennas in BS
N_ms = 1
cr = 1/2 # compressive ratio, (pilot length)/(N_BS)
mtx_v = N_BS # pilot matrix's shape is mtx_v x mtx_h
mtx_h = int(N_BS*cr)
output_dim = int(N_BS*cr)

In [9]:
SNR = [0, 5, 10, 15, 20]

In [3]:
# SNR = [0, 10, 20]
# data loading
train = 'H_train.mat'
val = 'H_val.mat'
test = 'H_test.mat'

mat = scipy.io.loadmat(train)
x_train = mat['H_train']
x_train = x_train.astype('float32')

mat = scipy.io.loadmat(val)
x_val = mat['H_val']
x_val = x_val.astype('float32')

mat = scipy.io.loadmat(test)
x_test = mat['H_test']
x_test = x_test.astype('float32')

In [4]:
train_dl=data.DataLoader(x_train, batch_size=128, shuffle=True)
val_dl=data.DataLoader(x_val, batch_size=128, shuffle=True)

In [11]:
class channel_estimation(nn.Module):
    def __init__(self, init_weights=True):
        super().__init__()
        self.output_dim=output_dim
        self.SN=nn.Linear(N_BS, output_dim, bias=False) 
        self.decode=nn.Linear(output_dim, N_BS, bias=False)
        
        self.f1=nn.Sequential(
            nn.Conv2d(2, 2, 3, padding=1),
            nn.BatchNorm2d(2),
            nn.LeakyReLU()
            )
        self.f2=nn.Sequential(
            nn.Conv2d(2, 4, 3, padding=1),
            nn.BatchNorm2d(4),
            nn.LeakyReLU()
            )
        self.f3=nn.Sequential(
            nn.Conv2d(4, 8, 3, padding=1),
            nn.BatchNorm2d(8),
            nn.LeakyReLU()
            )
        self.f4=nn.Sequential(
            nn.Conv2d(8, 16, 3, padding=1),
            nn.BatchNorm2d(16),
            nn.LeakyReLU()
            )
        self.f5=nn.Sequential(
            nn.Conv2d(16, 32, 3, padding=1),
            nn.BatchNorm2d(32),
            nn.LeakyReLU()
            )
        self.f6=nn.Sequential(
            nn.Conv2d(32, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU()
            )
        self.f7=nn.Sequential(
            nn.Conv2d(64, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU()
            )
        self.f8=nn.Sequential(
            nn.Conv2d(128, 256, 3, padding=1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU()
            )
        self.f9=nn.Sequential(
            nn.Conv2d(256, 2, 3, padding=1),
            nn.BatchNorm2d(2)
            )
        self.shortcut=nn.Sequential()
        self.LReLU=nn.LeakyReLU()
        if init_weights:
            self._initialize_weights()
    def forward(self, x):
        y=self.SN(x)
#         print('signal_shape')
#         print(y.shape)
        with torch.no_grad():
        
            sig_power=torch.mean(y**2, axis=1, keepdims=True)
    #         print(sig_power.shape)
            noise_var=torch.sqrt(sig_power/(10**(snr/10))).to(device)
            noise =torch.normal(0,1,size=y.shape).to(device)
            noise=noise_var*noise
    #         print('noise')
    #         print(noise.shape)
    #         y_t=torch.sum(y**2, axis=1)
    #         y_t=torch.sum(y_t, axis=1)
    #         n_t=torch.sum(noise**2, axis=1)
    #         n_t=torch.sum(n_t, axis=1)
    #         snr_t=torch.mean(y_t/n_t)
    #         print((y_t/n_t).shape)
    #         print(10* math.log10(snr_t))
        y=y+noise
        x=self.decode(y)
        x=x.reshape(x.shape[0], 2, m,n)
        t=x # shortcut
        x=self.f1(x)
        x=self.f2(x)
        x=self.f3(x)
        x=self.f4(x)
        x=self.f5(x)
        x=self.f6(x)
        x=self.f7(x)
        x=self.f8(x)
        x=self.f9(x)+self.shortcut(t)
        x=self.LReLU(x)
        x=x.reshape(x.shape[0],2,N_BS)
        return x
        
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
  

In [15]:
snr=0
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model=channel_estimation().to(device)
x=torch.randn(200, 2,256).to(device)
output=model(x)
print(output.size())

torch.Size([200, 2, 256])


In [12]:
def metric_batch(output, target): 
    # NMSE metric
    x_re=target[:,0,:]
    x_im=target[:,1,:]
    x_t=x_re+1j*x_im
    
    x_h_re=output[:,0,:]
    x_h_im=output[:,1,:]
    x_h_t=x_h_re+1j*x_h_im
    
    power=torch.sum(abs(x_t)**2, axis=1)
    mse=torch.sum(abs(x_t-x_h_t)**2, axis=1)    
    result=10*math.log10(torch.mean(mse/power))
    return result

def loss_batch(loss_func, output, target, opt=None):
    loss=loss_func(output, target)
    metric_b=metric_batch(output, target)

    if opt is not None:
        opt.zero_grad()
        loss.backward()
        opt.step()
    return loss.item(), metric_b

def loss_epoch(model, loss_func, dataset_dl, sanity_check=False, opt=None):
    running_loss=0.0
    running_metric=0.0
    len_data=len(dataset_dl.dataset)
    
    for xb in dataset_dl:
        xb=xb.float()
        xb=xb.to(device)
        output=model(xb)   
        loss_b, metric_b=loss_batch(loss_func, output, xb, opt)
        running_loss+=loss_b

        if metric_b is not None:
            running_metric+=metric_b

        if sanity_check is True:
            break
    loss=running_loss/(len_data/128)
    metric=running_metric/(len_data/128)

    return loss, metric

def train_val(model, params):
    num_epochs=params['num_epochs']
    loss_func=params["loss_func"]
    opt=params["optimizer"]
    train_dl=params["train_dl"]
    val_dl=params["val_dl"]
    sanity_check=params["sanity_check"]
    path2weights=params["path2weights"]

    loss_history={'train': [], 'val': []}
    metric_history={'train': [], 'val': []}

    best_loss=float('inf')
    start_time=time.time()

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs-1))

        model.train()
        train_loss, train_metric=loss_epoch(model, loss_func, train_dl, sanity_check, opt)
        loss_history['train'].append(train_loss)
        metric_history['train'].append(train_metric)

        model.eval()
        with torch.no_grad():
            val_loss, val_metric=loss_epoch(model, loss_func, val_dl, sanity_check)
            loss_history['val'].append(val_loss)
            metric_history['val'].append(val_metric)

            if val_loss<best_loss:
                best_loss=val_loss
                print('Get best val_loss')
                torch.save(model.state_dict(), f'.\\models_torch\\{cr}_{snr}_inverse_bestloss.pt')
                torch.save(model,f'.\\models_torch\\params\\{cr}_{snr}_inverse_bestloss_model.pt')
            

            print('SNR is %d dB, %d th iteration, train loss: %.6f, val loss: %.6f, accuracy: %.2f, time: %.4f min' %(snr, iters,  train_loss, val_loss, val_metric, (time.time()-start_time/60)))
            print('-'*10)

    torch.save(model.state_dict(), f'.\\models_torch\\{cr}_{snr}_inverse.pt')
    torch.save(model,f'.\\models_torch\\params\\{cr}_{snr}_inverse_model.pt')
    print(f'SNR: {snr}, the {iters} th model is saved')

In [13]:
for snr in SNR:
    for iters in range(1):        
        device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        model=channel_estimation().to(device) # model initialization  
        loss_func = nn.MSELoss() # loss function 
        opt = optim.Adam(model.parameters(), lr=0.001) # Adam optimizer
        params_train={
            'num_epochs':300,
            'optimizer': opt,
            'loss_func': loss_func,
            'train_dl': train_dl,
            'val_dl': val_dl,
            'sanity_check': False,
            'path2weights': './models/weights.pt',
        }

        train_val(model, params_train)

Epoch 0/299
Get best val_loss
SNR is 0 dB, 0 th iteration, train loss: 0.333723, val loss: 0.306319, accuracy: -2.09, time: 1612601127.5321 min
----------
Epoch 1/299
Get best val_loss
SNR is 0 dB, 0 th iteration, train loss: 0.286622, val loss: 0.278870, accuracy: -2.49, time: 1612601230.1004 min
----------
Epoch 2/299
Get best val_loss
SNR is 0 dB, 0 th iteration, train loss: 0.256482, val loss: 0.251062, accuracy: -2.93, time: 1612601332.7651 min
----------
Epoch 3/299
Get best val_loss
SNR is 0 dB, 0 th iteration, train loss: 0.229899, val loss: 0.223996, accuracy: -3.42, time: 1612601435.2900 min
----------
Epoch 4/299
Get best val_loss
SNR is 0 dB, 0 th iteration, train loss: 0.207296, val loss: 0.206574, accuracy: -3.74, time: 1612601537.8037 min
----------
Epoch 5/299
Get best val_loss
SNR is 0 dB, 0 th iteration, train loss: 0.188453, val loss: 0.185364, accuracy: -4.22, time: 1612601640.3594 min
----------
Epoch 6/299
Get best val_loss
SNR is 0 dB, 0 th iteration, train loss:

Get best val_loss
SNR is 0 dB, 0 th iteration, train loss: 0.116062, val loss: 0.125143, accuracy: -5.88, time: 1612606971.3790 min
----------
Epoch 58/299
SNR is 0 dB, 0 th iteration, train loss: 0.116034, val loss: 0.128537, accuracy: -5.75, time: 1612607073.9691 min
----------
Epoch 59/299
Get best val_loss
SNR is 0 dB, 0 th iteration, train loss: 0.115879, val loss: 0.124695, accuracy: -5.89, time: 1612607176.5533 min
----------
Epoch 60/299
SNR is 0 dB, 0 th iteration, train loss: 0.115848, val loss: 0.125035, accuracy: -5.88, time: 1612607279.1053 min
----------
Epoch 61/299
SNR is 0 dB, 0 th iteration, train loss: 0.115784, val loss: 0.126196, accuracy: -5.85, time: 1612607381.6874 min
----------
Epoch 62/299
SNR is 0 dB, 0 th iteration, train loss: 0.115693, val loss: 0.125714, accuracy: -5.86, time: 1612607484.2949 min
----------
Epoch 63/299
SNR is 0 dB, 0 th iteration, train loss: 0.115722, val loss: 0.125014, accuracy: -5.87, time: 1612607586.8467 min
----------
Epoch 64/29

SNR is 0 dB, 0 th iteration, train loss: 0.113988, val loss: 0.126169, accuracy: -5.84, time: 1612613021.0440 min
----------
Epoch 117/299
SNR is 0 dB, 0 th iteration, train loss: 0.114076, val loss: 0.128214, accuracy: -5.78, time: 1612613123.5548 min
----------
Epoch 118/299
SNR is 0 dB, 0 th iteration, train loss: 0.113899, val loss: 0.125446, accuracy: -5.87, time: 1612613226.1426 min
----------
Epoch 119/299
SNR is 0 dB, 0 th iteration, train loss: 0.113911, val loss: 0.125837, accuracy: -5.84, time: 1612613328.6121 min
----------
Epoch 120/299
SNR is 0 dB, 0 th iteration, train loss: 0.113937, val loss: 0.124593, accuracy: -5.90, time: 1612613431.0701 min
----------
Epoch 121/299
SNR is 0 dB, 0 th iteration, train loss: 0.113961, val loss: 0.125371, accuracy: -5.88, time: 1612613533.5893 min
----------
Epoch 122/299
SNR is 0 dB, 0 th iteration, train loss: 0.113929, val loss: 0.125289, accuracy: -5.87, time: 1612613636.0711 min
----------
Epoch 123/299
SNR is 0 dB, 0 th iteration

SNR is 0 dB, 0 th iteration, train loss: 0.113084, val loss: 0.126041, accuracy: -5.86, time: 1612619071.7247 min
----------
Epoch 176/299
SNR is 0 dB, 0 th iteration, train loss: 0.113135, val loss: 0.126530, accuracy: -5.81, time: 1612619174.3123 min
----------
Epoch 177/299
SNR is 0 dB, 0 th iteration, train loss: 0.113075, val loss: 0.125360, accuracy: -5.89, time: 1612619276.9723 min
----------
Epoch 178/299
SNR is 0 dB, 0 th iteration, train loss: 0.113042, val loss: 0.125254, accuracy: -5.88, time: 1612619379.5667 min
----------
Epoch 179/299
SNR is 0 dB, 0 th iteration, train loss: 0.112963, val loss: 0.129582, accuracy: -5.74, time: 1612619482.1472 min
----------
Epoch 180/299
SNR is 0 dB, 0 th iteration, train loss: 0.113087, val loss: 0.125778, accuracy: -5.84, time: 1612619584.7838 min
----------
Epoch 181/299
SNR is 0 dB, 0 th iteration, train loss: 0.112967, val loss: 0.126910, accuracy: -5.83, time: 1612619687.3098 min
----------
Epoch 182/299
SNR is 0 dB, 0 th iteration

SNR is 0 dB, 0 th iteration, train loss: 0.112405, val loss: 0.125659, accuracy: -5.86, time: 1612625124.0025 min
----------
Epoch 235/299
SNR is 0 dB, 0 th iteration, train loss: 0.112392, val loss: 0.128563, accuracy: -5.77, time: 1612625226.5317 min
----------
Epoch 236/299
SNR is 0 dB, 0 th iteration, train loss: 0.112350, val loss: 0.124504, accuracy: -5.92, time: 1612625329.0605 min
----------
Epoch 237/299
SNR is 0 dB, 0 th iteration, train loss: 0.112377, val loss: 0.125335, accuracy: -5.89, time: 1612625431.6570 min
----------
Epoch 238/299
SNR is 0 dB, 0 th iteration, train loss: 0.112392, val loss: 0.124872, accuracy: -5.90, time: 1612625534.2570 min
----------
Epoch 239/299
SNR is 0 dB, 0 th iteration, train loss: 0.112419, val loss: 0.125075, accuracy: -5.90, time: 1612625636.8237 min
----------
Epoch 240/299
SNR is 0 dB, 0 th iteration, train loss: 0.112387, val loss: 0.128038, accuracy: -5.78, time: 1612625739.3539 min
----------
Epoch 241/299
SNR is 0 dB, 0 th iteration

KeyboardInterrupt: 

In [14]:
loss_history={'test_0': [], 'test_5': [], 'test_10': [], 'test_20': []}
metric_history={'test_0': [], 'test_5': [], 'test_10': [], 'test_20': []}
sanity_check=False

SNR=[5]
for snr in SNR:
    for iters in range(1):   
        device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        model=channel_estimation().to(device) # model initialization  
        model.load_state_dict(torch.load( f'.\\models_torch\\{cr}_{snr}.pt'))
        model.eval()
        test_dl=data.DataLoader(x_test, batch_size=128, shuffle=False)

        with torch.no_grad():
            test_loss, test_metric=loss_epoch(model, loss_func, test_dl, sanity_check)
            loss_history[f'test_{snr}'].append(test_loss)
            metric_history[f'test_{snr}'].append(test_metric)

FileNotFoundError: [Errno 2] No such file or directory: '.\\models_torch\\0.5_5.pt'

In [13]:
metric_history

{'test_0': [-6.098152090307767],
 'test_10': [-12.26807310797302],
 'test_20': [-15.1880841676938]}